In [1]:
# import libraries
%run StatisticalFMEvaluator.ipynb    

In [5]:
caseStudyName = 'suv'

# read specification text
spec_processed0 = open("Mazda.txt","r", encoding='utf-8')
spec_processed0 = spec_processed0.read().replace("\r", "").replace("\n", "")
spec_processed1 = open("Mercedes.txt","r", encoding='utf-8')
spec_processed1 = spec_processed1.read().replace("\r", "").replace("\n", "")
spec_processed2 = open("Toyota.txt","r", encoding='utf-8')
spec_processed2 = spec_processed2.read().replace("\r", "").replace("\n", "")
spec_processed3 = open("Infinity.txt","r", encoding='utf-8')
spec_processed3 = spec_processed3.read().replace("\r", "").replace("\n", "")
spec_processed4 = open("Honda.txt","r", encoding='utf-8')
spec_processed4 = spec_processed4.read().replace("\r", "").replace("\n", "")
spec_processed5 = open("Audi.txt","r")
spec_processed5 = spec_processed5.read().replace("\r", "").replace("\n", "")

spec_processed = []
spec_processed.append(spec_processed0)
spec_processed.append(spec_processed1)
spec_processed.append(spec_processed2)
spec_processed.append(spec_processed3)
spec_processed.append(spec_processed4)
spec_processed.append(spec_processed5)

# invoke FM evaluator
fme = StatisticalFMEvaluator(spec_processed, caseStudyName)
fme.generateCorpus()
# fme.printCorpus()
# fme.printDictionary()
scores = fme.computeTfIdfScores()
# words_dict = fme.printDictionary()
print(scores)

####################### DOCUMENT ####################
####################### DOCUMENT ####################
####################### DOCUMENT ####################
####################### DOCUMENT ####################
####################### DOCUMENT ####################
####################### DOCUMENT ####################
{'maintain': 151.89024050196065, 'authorized': 108.99264188912886, 'ensure': 108.04899240413982, 'expert': 107.99819746833877, 'vehicle': 92.11861817908094, 'help': 90.09108842807117, 'word': 88.2951290386988, 'options': 71.28655815985611, 'specialized': 70.89551785756282, 'modify': 63.26502552616615, 'printing': 55.19585397425838, 'recommend': 45.8422796592992, 'result': 44.308216311510805, 'range': 43.5328946178907, 'improvement': 43.30761638101669, 'press': 43.23486726973752, 'engine': 42.72126142248682, 'air': 39.063811573718944, 'repairer': 38.99091200330361, 'damage': 36.7185658256584, 'index': 36.66713589689619, 'maintenance': 36.610083709171455, 'highly': 36.3

In [6]:
# Selecting 'products' as the root feature
root_feature_word = 'authorized'
# Compute all probable feature phrases
# starting with selected root feature
ngramDef = 2 # selecting phrases with +/- 2 words around the selected word 
# get all feature terms with a maximum of 5-grams
features = fme.getFeatureTermsList(root_feature_word, ngramDef)
allFeatures = fme.getAllFeaturesLst()
allTags = fme.getAllFeaturesTagsLst()

sentenceTags = fme.getDocTagsArray()

featureValues = [(allFeatures[idx],allTags[idx]) for idx in features]
# print(featureValues)

# get regression data
ra = RegressionAnalysis(features, allFeatures, allTags)
regressionData = ra.computeRegressionData()
regressionData.to_csv(caseStudyName+'_regressionData.csv', index = False, sep=',', encoding='utf-8')

# At this point pause the code execution and update the csv file by marking all features from the domain expert FM

['Feature', 'VBP', 'VB', 'NNS', 'VBN', 'VBG', 'NNPS', 'VBD', 'NNP', 'NN', 'VBZ']


In [7]:
print(allFeatures)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
print(sentenceTags)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# Feature Predictor Model
ra.modelGeneration_Coefficients(caseStudyName+'_regressionData.csv') # generate the coefficients

{'VBP': 0.9020388551733194, 'VB': 0.1513577952794785, 'NNS': 0.039396105755351914, 'VBN': 0.7862364682629526, 'VBG': -0.4766976170384055, 'NNPS': -0.004256070100532984, 'VBD': 0.7091326597568259, 'NNP': -0.5419138574400871, 'NN': 0.12907069655145545, 'VBZ': 0.10388511507219998}


In [10]:
# Relations Predictor
rp = RelationsPredictor(root_feature_word, allFeatures)
Rt = rp.computeDistanceScores()
print(Rt)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
# Evaluate FEATURE-X FM
# Read the input file with FM description
fm1 = open("FX-Suv-FM.txt","r")
fm1 = fm1.readlines()

evalResult = [] # Store the result lines, which will be used for generating the heat map
fm_features = {} # Store the feature phrase indexes that correspond to the phrases in the original text
relationsStart = 0 # pointer for the relations start in the FM text

allWrds = [tup[0] for tup in sentenceTags[0]]
allTags = [tup[1] for tup in sentenceTags[0]] 

# print(allWrds)

for lidx, line in enumerate(fm1):
    if line.isspace():
        relationsStart = lidx + 1
        break
    ph = line.split('\t')[1].strip('\n') # get the phrase
    print(ph)
    wrds = ph.split(' ') # get the words
    featureTagSeq = []
    
    # check if the phrase is a subsequence in the list of all words
    seq_idx = ra.isSeqIn(wrds, allWrds)
    print(seq_idx)
    if seq_idx != -1:
        for j in range(seq_idx, seq_idx+len(wrds)):
            featureTagSeq.append(allTags[j])
            # print("j =" + str(j))
            # print(allTags[j])
    else:
        foundTags = fme.getSimilarMeaningListOfWords(wrds)
        if foundTags:
            featureTagSeq = foundTags
            
    if not featureTagSeq:
        evalResult.append(str(lidx+1) + '\t' + ph.replace(' ','_') + '\t' + '0')
        fm_features[lidx+1] = -1
    else:
        evalResult.append(str(lidx+1) + '\t' + ph.replace(' ','_') + '\t' + str(ra.PredictFeatureProbability(featureTagSeq)))
        fm_features[lidx+1] = fme.getFeatureIndex(ph)

# Update the FM file with Feature evaluation
with open('FX-Suv-FM-Feval.txt','w') as f:
    f.write('\n'.join(evalResult))
    
# Evaluate relations
allFmRelations = []
for senIdx in range(relationsStart, len(fm1)):
    relation1 = fm1[senIdx].strip('\n').split('\t')
    f1 = relation1[0]
    f2 = relation1[1]
    rType = relation1[2]
    rEvalType = 'O'
    relScore = 0
    if fm_features[int(f1)] != -1 and fm_features[int(f2)] != -1:
        f1_score = Rt[int(fm_features[int(f1)])]
        f2_score = Rt[int(fm_features[int(f2)])]
        
        if f1_score >= 0.900 and f2_score >= 0.900:
            rEvalType = 'M'
            
        relScore = f1_score * f2_score            
    else:
        relScore = '0'
        
    allFmRelations.append(f1 + '\t' + f2 + '\t' + rType + '\t' + str(relScore) + '\t' + rEvalType)
 
 # Update the FM file with Feature evaluation
with open('FX-Suv-FM-Feval.txt','a') as f:
    f.write('\n\n')
    f.write('\n'.join(allFmRelations))

   vehicle
0
   accident
0
   adjustment
0
   child restraint
0
   front passenger seat
0
   journey
0
   protective function
0
   airbag function
0
   restraint system
0
  vehicle system
0
  seat belt seat
0
  switch
0
  tire pressure
0
  accident caused
0
  differ
0
  account
0
  route option
0
  activated auto
0
  automatic software update
0
  activation code
0
  storage medium
0
  activation con
0
  possible accelera tion
0
  standstill
0
  active profile
0
  vehicle display
0
  addi tion
0
  underneath
0
  additional injury
0
  seat belt
0
  additional online service
0
  navigation map
0
  map service
0
  online map
0
  additional value
0
  display content
0
  fuel filler flap location
0
  adjacent lane
0
  vehicle detected
0
  adjust
0
  adjustment function
0
  following situation
0
  accident situation
0
  traffic situation
0
  situation function
0
  excessive tire pressure
0
  airbag deployed
0
  minor deformation
0
  deployed airbag
0
  airbag deployment
0
  injury
0
  airbag 

 maximum tire pressure
0
 maximum weight
0
 trailer cable
0
 tire weight
0
 trailer drawbar noseweight
0
 memory function position switch
0
 memory pre set
0
 next track
0
 ment position
0
 wiper arm
0
 automatic mode
0
 climate mode
0
 air mode
0
 mode function
0
 mobile communication
0
 mobile phone
0
 phone mode
0
 transmission volume
0
 telephone multimedia
0
 connected telephone
0
 mobile phone connection
0
 code shown
0
 vehicle connection
0
 profile
0
 mobile phone multimedia
0
 mobile phone receptacle
0
 modification
0
 move ment
0
 moving part
0
 station
0
 music genre
0
 navigation announcement multimedia
0
 nec essary
0
 nect service
0
 net book
0
 net work operator
0
 network connection
0
 communi cation module
0
 online connection
0
 connection quality
0
 nominal value
0
 object
0
 on-board com puter
0
 inner segment
0
 outer segment
0
 on-board computer on-board
0
 on/off on/off
0
 online portal
0
 available vehicle
0
 online voice control online
0
 following online funct

 mobile phone network
0
 phone call
0
 mobile phone stow age compartment
0
 mobile phone stowage compartment caused
0
 model engine
0
 possible restriction
0
 multifunction
0
 network name
0
 next route
0
 oil note
0
 online function
0
 operating instruction
0
 optical warning
0
 overloa ded
0
 panoramic sliding sunroof
0
 completed
0
 personalization service
0
 please contact
0
 program
0
 subjected
0
 qualified spe cialist workshop
0
 radar sensor
0
 radiator fan
0
 rear wheel
0
 recom mended
0
 red area
0
 red warning display
0
 red distance
0
 red lock verification indicator
0
 reinforced
0
 disconnected
0
 respective service provider
0
 retrea ded
0
 rim checked
0
 roller dyna
0
 route list
0
 route overview
0
 route setting
0
 safety
0
 safety note
0
 breaking-in note
0
 search light
0
 seat adjustment
0
 seat adjustment switch
0
 seat cushion length adjust
0
 seat cushion inclination
0
 seat protected
0
 selected list
0
 station list
0
 station shown
0
 audible alarm
0
 tow-away

In [11]:
# Evaluate LITERATURE FM
fm1 = open("Lit-Suv-FM.txt","r")
fm1 = fm1.readlines()

evalResult = [] # Store the result lines, which will be used for generating the heat map
fm_features = {} # Store the feature phrase indexes that correspond to the phrases in the original text
relationsStart = 0 # pointer for the relations start in the FM text

allWrds = [tup[0] for tup in sentenceTags[0]]
allTags = [tup[1] for tup in sentenceTags[0]] 

for lidx, line in enumerate(fm1):
    if line.isspace():
        relationsStart = lidx + 1
        break
    ph = line.split('\t')[1].strip('\n') # get the phrase
    wrds = ph.split(' ') # get the words
    featureTagSeq = []
    
    # check if the phrase is a subsequence in the list of all words
    seq_idx = ra.isSeqIn(wrds, allWrds)
    # print(seq_idx)
    if seq_idx != -1:
        for j in range(seq_idx, seq_idx+len(wrds)):
            featureTagSeq.append(allTags[j])
            # print("j =" + str(j))
            # print(allTags[j])
    else:
        foundTags = fme.getSimilarMeaningListOfWords(wrds)
        if foundTags:
            featureTagSeq = foundTags
            
    if not featureTagSeq:
        evalResult.append(str(lidx+1) + '\t' + ph.replace(' ','_') + '\t' + '0')
        fm_features[lidx+1] = -1
    else:
        evalResult.append(str(lidx+1) + '\t' + ph.replace(' ','_') + '\t' + str(ra.PredictFeatureProbability(featureTagSeq)))
        fm_features[lidx+1] = fme.getFeatureIndex(ph)

# Update the FM file with Feature evaluation
with open('Lit-Suv-FM-Feval.txt','w') as f:
    f.write('\n'.join(evalResult))
    
# Evaluate relations
allFmRelations = []
for senIdx in range(relationsStart, len(fm1)):
    relation1 = fm1[senIdx].strip('\n').split('\t')
    f1 = relation1[0]
    f2 = relation1[1]
    rType = relation1[2]
    rEvalType = 'O'
    relScore = 0
    if fm_features[int(f1)] != -1 and fm_features[int(f2)] != -1:
        f1_score = Rt[int(fm_features[int(f1)])]
        f2_score = Rt[int(fm_features[int(f2)])]
        
        if f1_score >= 0.900 and f2_score >= 0.900:
            rEvalType = 'M'
            
        relScore = f1_score * f2_score            
    else:
        relScore = '0'
        
    allFmRelations.append(f1 + '\t' + f2 + '\t' + rType + '\t' + str(relScore) + '\t' + rEvalType)
 
 # Update the FM file with Feature evaluation
with open('Lit-Suv-FM-Feval.txt','a') as f:
    f.write('\n\n')
    f.write('\n'.join(allFmRelations))